<a href="https://colab.research.google.com/github/satya12388/Satya_tensorflow/blob/main/Classification/HandsOnML_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Calssification Model on Titanic Data Set

## Importing of Data and EDA

In [ ]:
import pandas as pd
Data = pd.read_excel("./titanic3.xls")

In [ ]:
Data.head(2)

,pclass,survived,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1,1,"Allen, Miss. Elisabeth Walton",female,29.0000,0,0,24160,211.3375,B5,S,2,NaN,"St Louis, MO"
1,1,1,"Allison, Master. Hudson Trevor",male,0.9167,1,2,113781,151.5500,C22 C26,S,11,NaN,"Montreal, PQ / Chesterville, ON"


In [ ]:
# Removing features for better analysis
Data.drop(columns=['name','cabin','embarked','boat','body','home.dest','ticket'],inplace = True)
Data.head()

,pclass,survived,sex,age,sibsp,parch,fare
0,1,1,female,29.0000,0,0,211.3375
1,1,1,male,0.9167,1,2,151.5500
2,1,0,female,2.0000,1,2,151.5500
3,1,0,male,30.0000,1,2,151.5500
4,1,0,female,25.0000,1,2,151.5500


In [ ]:
Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1309 entries, 0 to 1308
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   pclass    1309 non-null   int64  
 1   survived  1309 non-null   int64  
 2   sex       1309 non-null   object 
 3   age       1046 non-null   float64
 4   sibsp     1309 non-null   int64  
 5   parch     1309 non-null   int64  
 6   fare      1308 non-null   float64
dtypes: float64(2), int64(4), object(1)
memory usage: 71.7+ KB


In [ ]:
Data.corr(numeric_only=True)['survived']

pclass     -0.312469
survived    1.000000
age        -0.055513
sibsp      -0.027825
parch       0.082660
fare        0.244265
Name: survived, dtype: float64

## Spliting the Data to train and test

In [ ]:
from sklearn.model_selection import train_test_split
Train,Test = train_test_split(Data,test_size = 0.2)
print(Train.shape,Test.shape)

(1047, 7) (262, 7)


In [ ]:
Train_features = Train.drop('survived',axis =1)
Train_Labels = Train['survived']
print(Train_features.shape,Train_Labels.shape)

(1047, 6) (1047,)


## Preprocessing of Data

In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_selector,make_column_transformer
import numpy as np

num_pipeline = make_pipeline((SimpleImputer(strategy = 'median')),
                              (StandardScaler())
                              )

cat_pipeline = make_pipeline((SimpleImputer(strategy = 'most_frequent')),
                             (OneHotEncoder()))

preprocessed = make_column_transformer((num_pipeline,make_column_selector(dtype_include=np.number)),
                                       (cat_pipeline,make_column_selector(dtype_include=object)))

In [ ]:
Pre_train_features = preprocessed.fit_transform(Train_features)

In [ ]:
dummy = pd.DataFrame(Pre_train_features,columns = preprocessed.get_feature_names_out())
dummy.head()

,pipeline-1__pclass,pipeline-1__age,pipeline-1__sibsp,pipeline-1__parch,pipeline-1__fare,pipeline-2__sex_female,pipeline-2__sex_male
0,-0.341165,-0.439407,1.453139,-0.460232,-0.018368,0.0,1.0
1,-0.341165,-0.123690,-0.482531,-0.460232,-0.448263,0.0,1.0
2,0.853483,0.113098,-0.482531,-0.460232,-0.509775,0.0,1.0
3,0.853483,1.139180,-0.482531,-0.460232,-0.505812,0.0,1.0
4,-1.535813,1.612756,-0.482531,-0.460232,-0.076003,1.0,0.0


## Model using Normal Logistic regression

In [ ]:
from sklearn.linear_model import LogisticRegression

log_model = make_pipeline(preprocessed,LogisticRegression())
log_model.fit(Train_features,Train_Labels)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7b06d7b37ca0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7b06d7643dc0>)])),
                ('logisticregression', LogisticRegression())])

In [ ]:
from sklearn.model_selection import cross_val_predict
preds = cross_val_predict(log_model,Train_features,Train_Labels,cv = 3)
preds.shape

(1047,)

### Train Accuracy and Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Train_Labels,preds,normalize = 'true')

array([[0.84853168, 0.15146832],
       [0.3275    , 0.6725    ]])

In [ ]:
from sklearn.metrics import f1_score
f1_score(Train_Labels,preds)

0.7014341590612777

### Test Predictions and Accuracy of Model

In [ ]:
test_preds = log_model.predict(Test.drop('survived',axis =1))

In [ ]:
test_preds.shape

(262,)

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(Test['survived'],test_preds,normalize = 'true')

array([[0.7962963, 0.2037037],
       [0.25     , 0.75     ]])

In [ ]:
f1_score(Test['survived'],test_preds)

0.721153846153846

## Model using SVM Classifiers

* Support vector machines try to clasify the data by seperating two extreme data for each class, those extreme data points are called Support Vectors

* For Regression they try to take the interior points and fit them to reduce the error

In [ ]:
from sklearn.svm import LinearSVC
LSVC_model = make_pipeline(preprocessed,LinearSVC())
LSVC_model.fit(Train_features,Train_Labels)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7b06d7b37ca0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7b06d7643dc0>)])),
                ('linearsvc', LinearSVC())])

In [ ]:
confusion_matrix(Train_Labels,LSVC_model.predict(Train_features),normalize='true')

array([[0.85625966, 0.14374034],
       [0.3375    , 0.6625    ]])

In [ ]:
from sklearn.svm import SVC
SVC_model = make_pipeline(preprocessed,SVC(C=3))
SVC_model.fit(Train_features,Train_Labels)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7b06d7b37ca0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7b06d7643dc0>)])),
                ('svc', SVC(C=3))])

In [ ]:
confusion_matrix(Train_Labels,SVC_model.predict(Train_features),normalize='true')

array([[0.8763524, 0.1236476],
       [0.295    , 0.705    ]])

In [ ]:
f1_score(Train_Labels,SVC_model.predict(Train_features))

0.7401574803149606

📒 We could see the SVC has better performed than Logistic Regression and Linear SVC

## Model using Ensemble methods

* Ensemble methods are group of pre trained models and we use them all to predict the output.
* For example an ensemble model may contain logistic regression, Decission tree or a SVM Classifier. we use all the models to predict the desired outcome, this is called `Voter Classifier`
* The other method is to train same algorithm say Logistic Classifier on different samplings of data, this is called `Bagging`
* If Bagging is done using Decission trees it is called `Random Forests`

In [ ]:
# model using voter Classifier
from sklearn.ensemble import VotingClassifier

VC_model = make_pipeline(preprocessed,VotingClassifier(estimators = [
    ('lr',LogisticRegression()),
    ('sv',SVC())]
    ))
VC_model.fit(Train_features,Train_Labels)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7b06d7b37ca0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7b06d7643dc0>)])),
                ('votingclassifier',
                 VotingClassifier(estimators=[('lr', LogisticRegression()),
                                              ('sv', SVC())]))])

In [ ]:
f1_score(Train_Labels,VC_model.predict(Train_features))

0.7166212534059946

In [ ]:
# Model using Bagging
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
Bag_model  = make_pipeline(preprocessed,
                           BaggingClassifier(estimator = DecisionTreeClassifier(),
                                             n_estimators = 20,
                                             oob_score = True))
Bag_model.fit(Train_features,Train_Labels)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7b06d7b37ca0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7b06d7643dc0>)])),
                ('baggingclassifier',
                 BaggingClassifier(estimator=DecisionTreeClassifier(),
                                   n_estimators=20, oob_score=True))])

In [ ]:
f1_score(Train_Labels,Bag_model.predict(Train_features))

0.949748743718593

⚒️ Ohooo,, since we beat every other model lets test this Bagging model on our test data set

In [ ]:
Test_features = Test.drop('survived',axis =1)
Test_Labels = Test['survived']

In [ ]:
Bag_model_preds = Bag_model.predict(Test_features)

In [ ]:
confusion_matrix(Test_Labels,Bag_model_preds,normalize = 'true')

array([[0.83333333, 0.16666667],
       [0.33      , 0.67      ]])

In [ ]:
f1_score(Test_Labels,Bag_model_preds)

0.6907216494845361

Hurray Thats 4 % increase in f1-score

## Model using Random Forests

In [ ]:
from sklearn.ensemble import RandomForestClassifier
RF_model = make_pipeline(preprocessed,
                         RandomForestClassifier()) # same as bagging classifier but with 100 Decision trees
RF_model.fit(Train_features,Train_Labels)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('pipeline-1',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='median')),
                                                                  ('standardscaler',
                                                                   StandardScaler())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7b06d7b37ca0>),
                                                 ('pipeline-2',
                                                  Pipeline(steps=[('simpleimputer',
                                                                   SimpleImputer(strategy='most_frequent')),
                                                                  ('onehotencoder',
                                                                   OneHotEncoder())]),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x7b06d7643dc0>)])),
                ('randomforestclassifier', RandomForestClassifier())])

In [ ]:
confusion_matrix(Train_Labels,RF_model.predict(Train_features),normalize = 'true')

array([[0.98299845, 0.01700155],
       [0.0625    , 0.9375    ]])

In [ ]:
# Test Data Predictions
RF_Preds = RF_model.predict(Test_features)

In [ ]:
confusion_matrix(Test_Labels,RF_Preds,normalize = 'true')

array([[0.79012346, 0.20987654],
       [0.31      , 0.69      ]])

In [ ]:
f1_score(Test_Labels,RF_Preds)

0.6798029556650246

## Introducing Neural Networks and their Fine Tuning Strategies

In [ ]:
# Checking shape of input
Pre_train_features.shape
pre_test_features = preprocessed.transform(Test_features)
pre_test_features.shape

(262, 7)

In [ ]:
import tensorflow as tf

tf.keras.utils.set_random_seed(42)
# building Base Model
Base_Model = tf.keras.Sequential([
    tf.keras.layers.Dense(10,activation = 'relu',input_shape = (7,)),
    tf.keras.layers.Dense(10,activation = 'relu'),
    tf.keras.layers.Dense(1,activation = 'sigmoid')
])

# Compiling the model
Base_Model.compile(optimizer = tf.keras.optimizers.Adam(),
                   loss = tf.keras.losses.BinaryCrossentropy(),
                   metrics = ['accuracy'])

#fit the model
Base_Model.fit(Pre_train_features,Train_Labels,
               epochs =5,
               validation_split=0.1)


Epoch 1/5
30/30 [==============================] - 2s 17ms/step - loss: 0.6156 - accuracy: 0.6890 - val_loss: 0.5556 - val_accuracy: 0.7714
Epoch 2/5
30/30 [==============================] - 0s 7ms/step - loss: 0.5866 - accuracy: 0.7113 - val_loss: 0.5232 - val_accuracy: 0.7714
Epoch 3/5
30/30 [==============================] - 0s 7ms/step - loss: 0.5653 - accuracy: 0.7197 - val_loss: 0.5002 - val_accuracy: 0.7714
Epoch 4/5
30/30 [==============================] - 0s 7ms/step - loss: 0.5479 - accuracy: 0.7335 - val_loss: 0.4842 - val_accuracy: 0.7714
Epoch 5/5
30/30 [==============================] - 0s 6ms/step - loss: 0.5324 - accuracy: 0.7611 - val_loss: 0.4678 - val_accuracy: 0.7810


⚒️ With just base model we have achieved a model with validation accuracy of nearly 78 %.. Now lets see some predictions

In [ ]:
from sklearn.metrics import precision_score,recall_score,f1_score

# Model Predictions, Confusion Matrix and f1 Score
base_model_preds = tf.round(Base_Model.predict(pre_test_features))

# Checking shapes of Test labels and Predictions
print(f'predctions shape: {tf.squeeze(base_model_preds).shape}') # Squeeze to match shape of test Labels
print(f'True Labels shape: {Test_Labels.shape}')

# Confusion Matrix
print('\n--- Confusion Matrix---\n')
print(confusion_matrix(Test_Labels,tf.squeeze(base_model_preds)))

# Precision, Recall,f1 Score
print('\n---Metrics are as below---')

print(f'precision: {precision_score(Test_Labels,tf.squeeze(base_model_preds))}')
print(f'ReCall: {recall_score(Test_Labels,tf.squeeze(base_model_preds))}')
print(f'F1_Score: {f1_score(Test_Labels,tf.squeeze(base_model_preds))}')

9/9 [==============================] - 0s 2ms/step
predctions shape: (262,)
True Labels shape: (262,)

--- Confusion Matrix---

[[148  14]
 [ 54  46]]

---Metrics are as below---
precision: 0.7666666666666667
ReCall: 0.46
F1_Score: 0.575


🔭 As an importan observation here model has done well on precission but failed on recall, precisson is the measure where out of predicted as yes how many were correct (`T.P/T.P+F.P`), Recall is the measure where out of true yes how many it is predicted as Yes(`T.P/T.P+F.N`)

Since it failed on recall, overall f1 score reduced to 58 %


### Lets try to `fine tune` model by improving overall f1 score or reducing false negatives

In [ ]:
# we will use Keras tuner to search the best hyper parameteers of the model
%pip install -q -U keras-tuner
import keras_tuner as KT

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.1/129.1 kB 2.4 MB/s eta 0:00:00


In [ ]:
# building model with lot of hyper parameters

def fine_model(hp):
  n_hidden = hp.Int('hidden layers',min_value = 0, max_value = 5, default = 2)
  n_neurons = hp.Int('neurons per layer',min_value = 5, max_value = 128,default = 10)
  lr = hp.Float('learning rate', min_value = 1e-4, max_value = 1e-1)
  optimizer = hp.Choice('optimizer',values = ['sgd','adam'])

  if optimizer == 'sgd':
    optimizer = tf.keras.optimizers.SGD(lr)
  else:
    optimizer = tf.keras.optimizers.Adam(lr)

  model = tf.keras.Sequential()
  for _ in range(n_hidden):
      model.add(tf.keras.layers.Dense(n_neurons,activation = 'relu'))

  model.add(tf.keras.layers.Dense(1,activation = 'sigmoid'))

  model.compile(optimizer = optimizer,
                loss= tf.keras.losses.BinaryCrossentropy(),
                metrics = ['accuracy'])

  return model

In [ ]:
obj = KT.Objective('val_accuracy',direction = 'max')
rs_model_tuner =  KT.RandomSearch(fine_model, objective = obj, max_trials = 5,overwrite = True)

In [ ]:
rs_model_tuner.search(Pre_train_features,Train_Labels,
                      epochs =5,
                      validation_split=0.1)

Trial 5 Complete [00h 00m 03s]
val_accuracy: 0.7809523940086365

Best val_accuracy So Far: 0.8476190567016602
Total elapsed time: 00h 00m 16s


In [ ]:
hyper_params = rs_model_tuner.get_best_hyperparameters(num_trials = 2)
for i in hyper_params:
  print(i.values)

{'hidden layers': 1, 'neurons per layer': 93, 'learning rate': 0.054348153163871954, 'optimizer': 'adam'}
{'hidden layers': 5, 'neurons per layer': 56, 'learning rate': 0.05513545291795473, 'optimizer': 'adam'}


In [ ]:
Best_models = rs_model_tuner.get_best_models(num_models=5)
top_model = Best_models[0]

In [ ]:
top_model.fit(Pre_train_features,Train_Labels,epochs=5,validation_split=0.1)

Epoch 1/5
30/30 [==============================] - 2s 20ms/step - loss: 0.4820 - accuracy: 0.7898 - val_loss: 0.3711 - val_accuracy: 0.8667
Epoch 2/5
30/30 [==============================] - 0s 6ms/step - loss: 0.4574 - accuracy: 0.8036 - val_loss: 0.3875 - val_accuracy: 0.8667
Epoch 3/5
30/30 [==============================] - 0s 6ms/step - loss: 0.4724 - accuracy: 0.7866 - val_loss: 0.4963 - val_accuracy: 0.8190
Epoch 4/5
30/30 [==============================] - 0s 6ms/step - loss: 0.4683 - accuracy: 0.7866 - val_loss: 0.3528 - val_accuracy: 0.8667
Epoch 5/5
30/30 [==============================] - 0s 7ms/step - loss: 0.4622 - accuracy: 0.7983 - val_loss: 0.4262 - val_accuracy: 0.8286


In [ ]:
top_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 93)                744       
                                                                 
 dense_1 (Dense)             (None, 1)                 94        
                                                                 
Total params: 838 (3.27 KB)
Trainable params: 838 (3.27 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
